In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import radians, sin, cos, sqrt, atan2
from itertools import combinations

# pyflamegpu
from pyflamegpu import pyflamegpu as fg

print("Librerías cargadas correctamente")


ModuleNotFoundError: No module named 'pyflamegpu'

In [ ]:
COUNTRY_DATA_PATH = "data/covid_19_country_daily_with_coords.csv"
GRAPH_DATA_PATH = "data/country_proximity_edges.csv"

country_df = pd.read_csv(COUNTRY_DATA_PATH, parse_dates=["ObservationDate"], dayfirst=False)
edge_df = pd.read_csv(GRAPH_DATA_PATH)

print(f"Registros país-fecha: {len(country_df):,}")
print(f"Aristas proximidad: {len(edge_df):,}")
country_df.head()


In [ ]:
# Lista ordenada de fechas y resumen final por país (última fecha disponible)
country_df = country_df.sort_values(["Country/Region", "ObservationDate"]).reset_index(drop=True)
unique_dates = country_df["ObservationDate"].sort_values().unique()
num_steps = len(unique_dates)
print(f"Fechas distintas en el dataset: {num_steps}")

latest_per_country = (
    country_df.sort_values("ObservationDate")
              .groupby("Country/Region", as_index=False)
              .tail(1)
              .reset_index(drop=True)
)
print(f"Países únicos: {len(latest_per_country):,}")
latest_per_country.head()


In [ ]:
# Construimos los estados iniciales (S, E, I, R, D) por país
min_active = 0.1  # evita valores exactamente cero

latest_per_country["Active"] = (
    latest_per_country["Confirmed"]
    - latest_per_country["Recovered"]
    - latest_per_country["Deaths"]
).clip(lower=0.0)

latest_per_country["Population_proxy"] = (
    latest_per_country["Confirmed"]
    + latest_per_country["Recovered"]
    + latest_per_country["Deaths"]
) * 1.25

latest_per_country["Population_proxy"] = latest_per_country.apply(
    lambda row: max(row["Population_proxy"], row["Active"] + row["Recovered"] + row["Deaths"] + 1.0),
    axis=1
)

latest_per_country["I0"] = latest_per_country["Active"].clip(lower=min_active)
latest_per_country["R0"] = latest_per_country["Recovered"].clip(lower=0.0)
latest_per_country["D0"] = latest_per_country["Deaths"].clip(lower=0.0)
latest_per_country["S0"] = (
    latest_per_country["Population_proxy"]
    - latest_per_country["I0"]
    - latest_per_country["R0"]
    - latest_per_country["D0"]
).clip(lower=0.0)
latest_per_country["E0"] = 0.0

state_cols = ["Country/Region", "Latitud_promedio", "Longitud_promedio", "Population_proxy", "S0", "E0", "I0", "R0", "D0"]
state_df = latest_per_country[state_cols].rename(columns={"Population_proxy": "Population"}).reset_index(drop=True)
state_df.head()


In [ ]:
# Construimos matrices auxiliares: IDs, poblaciones y proximidades normalizadas
state_df = state_df.sort_values("Country/Region").reset_index(drop=True)
num_countries = len(state_df)

country_to_id = {name: idx for idx, name in enumerate(state_df["Country/Region"])}
id_to_country = {idx: name for name, idx in country_to_id.items()}

coords = state_df[["Latitud_promedio", "Longitud_promedio"]].to_numpy(dtype=float)
population = state_df["Population"].to_numpy(dtype=float)

# Distancia Haversine pairwise
RADIUS_EARTH_KM = 6371.0

def haversine(lat1, lon1, lat2, lon2):
    lat1_rad, lon1_rad = radians(lat1), radians(lon1)
    lat2_rad, lon2_rad = radians(lat2), radians(lon2)
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = sin(dlat / 2) ** 2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return RADIUS_EARTH_KM * c

distance_matrix = np.zeros((num_countries, num_countries), dtype=float)
for i in range(num_countries):
    for j in range(i + 1, num_countries):
        dist = haversine(coords[i, 0], coords[i, 1], coords[j, 0], coords[j, 1])
        distance_matrix[i, j] = distance_matrix[j, i] = dist

max_dist = distance_matrix.max()
proximity_matrix = 1.0 - (distance_matrix / max_dist)
np.fill_diagonal(proximity_matrix, 0.0)

print(f"Países (agentes): {num_countries}")
print(f"Distancia máxima: {max_dist:,.2f} km")
state_df.head()


In [ ]:
# Parámetros epidemiológicos razonables (puedes ajustarlos luego)
params = {
    "beta_local": 0.30,       # tasa de contagio interna
    "beta_travel": 0.05,      # influencia de viajes entre países
    "sigma": 1.0 / 5.0,       # 1/incubación ≈ 5 días
    "gamma": 1.0 / 7.0,       # 1/duración infecciosa ≈ 7 días
    "mortality": 0.01,        # 1% de los infectados activos mueren al recuperarse
    "dt": 1.0                 # paso temporal = 1 día
}
params


In [ ]:
# Definición del modelo pyflamegpu
model = fg.ModelDescription("COVID_Country_Model")

country_agent = model.newAgent("Country")
country_agent.newVariableUInt("id")
country_agent.newVariableFloat("population")
country_agent.newVariableFloat("S")
country_agent.newVariableFloat("E")
country_agent.newVariableFloat("I")
country_agent.newVariableFloat("R")
country_agent.newVariableFloat("D")
country_agent.newVariableFloat("lat")
country_agent.newVariableFloat("lon")

message = model.newMessageBruteForce("InfectiousMsg")
message.newVariableUInt("id")
message.newVariableFloat("infectious")

output_func = model.newAgentFunction("output_infectious",
"""
FLAMEGPU_AGENT_FUNCTION(output_infectious, flamegpu::MessageNone, flamegpu::MessageBruteForce) {
    const unsigned int id = FLAMEGPU->getVariable<unsigned int>("id");
    const float I = FLAMEGPU->getVariable<float>("I");
    FLAMEGPU->message_out.setVariable<unsigned int>("id", id);
    FLAMEGPU->message_out.setVariable<float>("infectious", I);
    return flamegpu::ALIVE;
}
""")
output_func.setMessageOutput("InfectiousMsg")

update_func = model.newAgentFunction("update_state",
"""
FLAMEGPU_AGENT_FUNCTION(update_state, flamegpu::MessageBruteForce, flamegpu::MessageNone) {
    const unsigned int id = FLAMEGPU->getVariable<unsigned int>("id");
    const unsigned int num_agents = FLAMEGPU->environment.getProperty<unsigned int>("num_agents");

    float S = FLAMEGPU->getVariable<float>("S");
    float E = FLAMEGPU->getVariable<float>("E");
    float I = FLAMEGPU->getVariable<float>("I");
    float R = FLAMEGPU->getVariable<float>("R");
    float D = FLAMEGPU->getVariable<float>("D");
    const float population = FLAMEGPU->getVariable<float>("population");

    const float beta_local = FLAMEGPU->environment.getProperty<float>("beta_local");
    const float beta_travel = FLAMEGPU->environment.getProperty<float>("beta_travel");
    const float sigma = FLAMEGPU->environment.getProperty<float>("sigma");
    const float gamma = FLAMEGPU->environment.getProperty<float>("gamma");
    const float mortality = FLAMEGPU->environment.getProperty<float>("mortality");
    const float dt = FLAMEGPU->environment.getProperty<float>("dt");

    const float inv_pop = 1.0f / (population + 1.0f);
    float local_force = beta_local * I * inv_pop;

    float travel_force = 0.0f;
    for (const auto &msg : FLAMEGPU->message_in) {
        const unsigned int other_id = msg.getVariable<unsigned int>("id");
        if (other_id == id) continue;
        const float prox = FLAMEGPU->environment.getProperty<float>("proximity_matrix", id * num_agents + other_id);
        const float other_I = msg.getVariable<float>("infectious");
        float other_pop = FLAMEGPU->environment.getProperty<float>("population_array", other_id);
        if (other_pop < 1.0f) other_pop = 1.0f;
        travel_force += prox * (other_I / other_pop);
    }
    travel_force *= beta_travel;

    float new_exposed = (local_force + travel_force) * S * dt;
    if (new_exposed > S) new_exposed = S;

    float new_infected = sigma * E * dt;
    if (new_infected > E) new_infected = E;

    float total_progress = gamma * I * dt;
    if (total_progress > I) total_progress = I;
    float deaths = total_progress * mortality;
    float recoveries = total_progress - deaths;

    S -= new_exposed;
    E += new_exposed - new_infected;
    I += new_infected - recoveries - deaths;
    R += recoveries;
    D += deaths;

    if (S < 0.0f) S = 0.0f;
    if (E < 0.0f) E = 0.0f;
    if (I < 0.0f) I = 0.0f;
    if (R < 0.0f) R = 0.0f;

    FLAMEGPU->setVariable<float>("S", S);
    FLAMEGPU->setVariable<float>("E", E);
    FLAMEGPU->setVariable<float>("I", I);
    FLAMEGPU->setVariable<float>("R", R);
    FLAMEGPU->setVariable<float>("D", D);

    return flamegpu::ALIVE;
}
""")
update_func.setMessageInput("InfectiousMsg")

layer1 = model.newLayer("A_output")
layer1.addAgentFunction(output_func)
layer2 = model.newLayer("B_update")
layer2.addAgentFunction(update_func)

env = model.Environment()
env.newPropertyUInt("num_agents", num_countries)
env.newArrayFloat("proximity_matrix", num_countries * num_countries)
env.newArrayFloat("population_array", num_countries)
env.newPropertyFloat("beta_local", params["beta_local"])
env.newPropertyFloat("beta_travel", params["beta_travel"])
env.newPropertyFloat("sigma", params["sigma"])
env.newPropertyFloat("gamma", params["gamma"])
env.newPropertyFloat("mortality", params["mortality"])
env.newPropertyFloat("dt", params["dt"])

print("Modelo definido")


In [ ]:
# Construimos el vector inicial de agentes
init_population = fg.AgentVector(country_agent, num_countries)

for idx, row in state_df.iterrows():
    agent = init_population[idx]
    agent.setVariableUInt("id", idx)
    agent.setVariableFloat("population", row["Population"])
    agent.setVariableFloat("S", row["S0"])
    agent.setVariableFloat("E", row["E0"])
    agent.setVariableFloat("I", row["I0"])
    agent.setVariableFloat("R", row["R0"])
    agent.setVariableFloat("D", row["D0"])
    agent.setVariableFloat("lat", row["Latitud_promedio"])
    agent.setVariableFloat("lon", row["Longitud_promedio"])

print("Agentes inicializados")


In [ ]:
# Configuración de la simulación
simulation = fg.CUDASimulation(model)
simulation.SimulationConfig().random_seed = 42

environment_values = {
    "num_agents": num_countries,
    "beta_local": params["beta_local"],
    "beta_travel": params["beta_travel"],
    "sigma": params["sigma"],
    "gamma": params["gamma"],
    "mortality": params["mortality"],
    "dt": params["dt"]
}

for key, value in environment_values.items():
    simulation.setEnvironmentProperty(key, value)

simulation.setEnvironmentPropertyArray("proximity_matrix", proximity_matrix.flatten().astype(np.float32).tolist())
simulation.setEnvironmentPropertyArray("population_array", population.astype(np.float32).tolist())

simulation.setPopulationData(init_population)
print("Simulación configurada")


In [ ]:
def snapshot(simulation_obj):
    agent_vec = fg.AgentVector(country_agent)
    simulation_obj.getPopulationData(agent_vec)
    records = []
    for agent in agent_vec:
        idx = agent.getVariableUInt("id")
        records.append({
            "id": int(idx),
            "country": id_to_country[idx],
            "S": agent.getVariableFloat("S"),
            "E": agent.getVariableFloat("E"),
            "I": agent.getVariableFloat("I"),
            "R": agent.getVariableFloat("R"),
            "D": agent.getVariableFloat("D")
        })
    return records

print("Función snapshot lista")


In [ ]:
results = []

# Estado inicial (step = 0)
initial_records = snapshot(simulation)
for rec in initial_records:
    rec["step"] = 0
    rec["date"] = pd.NaT
    results.append(rec)

for step_idx, current_date in enumerate(unique_dates, start=1):
    simulation.step()
    step_records = snapshot(simulation)
    for rec in step_records:
        rec["step"] = step_idx
        rec["date"] = current_date
        results.append(rec)

results_df = pd.DataFrame(results)
print(f"Registros en resultados: {len(results_df):,}")
results_df.head()


In [ ]:
global_series = (
    results_df.groupby("step")["I"].sum()
               .reset_index()
               .rename(columns={"I": "Infectious_global"})
)

plt.figure(figsize=(10, 4))
plt.plot(global_series["step"], global_series["Infectious_global"], label="Infectados globales")
plt.xlabel("Paso (días)")
plt.ylabel("Personas")
plt.title("Evolución global de infectados (modelo)")
plt.grid(alpha=0.3)
plt.legend()
plt.show()


In [ ]:
latest_results = results_df[results_df["step"] == results_df["step"].max()]
top_countries = latest_results.nlargest(10, "I")[
    ["country", "S", "E", "I", "R", "D"]
]
print("Top 10 países por infectados al final de la simulación:")
top_countries


In [ ]:
SIM_OUTPUT_PATH = "data/covid_simulation_results.csv"
results_df.to_csv(SIM_OUTPUT_PATH, index=False)
print(f"Resultados completos guardados en {SIM_OUTPUT_PATH}")


## Notas finales
- El modelo es simplificado: S/E/I/R agregados por país, sin demografía.
- Podés ajustar los parámetros en `params` antes de re-ejecutar.
- La matriz de proximidades puede reemplazarse por valores custom (por ejemplo, del CSV de aristas) si querés experimentar.
- El archivo `data/covid_simulation_results.csv` guarda la evolución país-día de los estados.

Esto debería darte una base clara para continuar con experimentos más avanzados.


## Notas finales
- El modelo es simplificado: S/E/I/R agregados por país, sin demografía.
- Podés ajustar los parámetros en `params` antes de re-ejecutar.
- La matriz de proximidades puede reemplazarse por valores custom (por ejemplo, del CSV de aristas) si querés experimentar.
- El archivo `data/covid_simulation_results.csv` guarda la evolución país-día de los estados.

Esto debería darte una base clara para continuar con experimentos más avanzados.
